In [1]:
# DATA INPUT SCRIPT 1/2 {}
# THIS SCRIPT IS RUNNING SCRIPTS FROM. https://rapidapi.com/coingecko/api/coingecko?endpoint=apiendpoint_4125add1-6b8e-4d2d-ade7-f6d2275b6a9b
import os,sys,inspect
import importlib
import os.path
from datetime import datetime
# run this if you edit the import module
#importlib.reload(fileManagement)
current_path = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_path = os.path.dirname(current_path)
src_path = os.path.dirname(parent_path)
root_path = os.path.dirname(src_path)
env_path = root_path+'/env'
data_path = root_path+'/data'
sys.path.insert(0, src_path) 
# IMPORT CUSTOM MODULES (from ./common)
from common import fileManagement
# GLOBAL VARS
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
#PROGRAM
print('PATH VARIABLES CREATED')
print('src_path  :',src_path)
print('root_path :',root_path)
print('env_path  :',env_path)
print('data_path :',data_path)
print( current_time,'Utilty functions generated..')
print(current_time,'All code run in this block')


PATH VARIABLES CREATED
src_path  : /Users/paul/Documents/dev/python/crytpoCurrencyAnalysis/src
root_path : /Users/paul/Documents/dev/python/crytpoCurrencyAnalysis
env_path  : /Users/paul/Documents/dev/python/crytpoCurrencyAnalysis/env
data_path : /Users/paul/Documents/dev/python/crytpoCurrencyAnalysis/data
13:27:47 Utilty functions generated..


In [2]:
# A GET COINS
# 1. Get All the coins
# 2. Create a paged list of the coins (so that I can hit API with many small requests)
import requests
import json
# 1. Get All the coins
def getCoins():
    def createCoinList(jsonData):
        result = []
        for n in range(len(jsonData)):
            result.append(jsonData[n]['id'])
        return result
    def fetchData():
        url = "https://coingecko.p.rapidapi.com/coins/list"
        headers = {
            'x-rapidapi-key': rapidapiKey,
            'x-rapidapi-host': "coingecko.p.rapidapi.com"
            }
        response = requests.request("GET", url, headers=headers)
        jsonData = json.loads(response.text)
        return jsonData
    #RUN
    jsonData = fetchData()
    result = createCoinList(jsonData)
    return result
# 2. Create a paged list of the coins
def divide_chunks(myList, page_number):
    def yield_chunks(myList, page_number):
        # looping till length l
        for i in range(0, len(myList), page_number): 
            yield myList[i:i + page_number]
    result = list(yield_chunks(all_coins, page_number))
    print (current_time,'pagedCoins created: Pages Created:', len(result))
    return result
#PROGRAM
rapidapiKey = fileManagement.JSONImportEnv('x-rapidapi-key',env_path)
all_coins = getCoins()
outputPagedCoins = divide_chunks(all_coins,3)
fileManagement.JSONExport('pagedCoins.json',data_path,outputPagedCoins)
print(current_time,'All code run in this block')




13:27:47 Imported Env: /Users/paul/Documents/dev/python/crytpoCurrencyAnalysis/env/env.json
13:27:47 pagedCoins created: Pages Created: 2264
/Users/paul/Documents/dev/python/crytpoCurrencyAnalysis/data/pagedCoins.json
Records Exported: 2264 records
13:27:47 Exported file: /Users/paul/Documents/dev/python/crytpoCurrencyAnalysis/data/pagedCoins.json File: Overwritten


In [3]:
# B. GET STATS 
import time
import random

def createQueryString(myList):# convert list format: ['bitcoin','etherium'] to string: 'bitcoin,etherium'
    #used in fetchStatsAll()
    def removeUnwantedChars(myString):
        no_leftBrackets =  myString.replace("[", "")
        no_rightBrackets = no_leftBrackets.replace("]", "")
        no_quotes = no_rightBrackets.replace("'", "")
        result = no_quotes
        return result
    myString = str(myList)
    result = removeUnwantedChars(myString)
    return result
def fetchStats(coins):
    #used in fetchStatsAll()
    url = "https://coingecko.p.rapidapi.com/coins/markets"
    querystring = {"vs_currency":"usd","ids":coins,"per_page":"250","order":"market_cap_desc"}
    headers = {
        'x-rapidapi-key': rapidapiKey,
        'x-rapidapi-host': "coingecko.p.rapidapi.com"
        }
    response = requests.request("GET", url, headers=headers, params=querystring)
    jsonData = json.loads(response.text)
    return jsonData
def fetchStatsAll(pagedCoins): #run fetchStats() n times
    #fetch the stats for the coins 1 page at a time (to overcome API limit)
    fetchedCollated=[]
    print('Total Pages:', len(pagedCoins))
    #number_of_Pages = len(pagedCoins)
    number_of_pages = 3
    for pageNumber in range(number_of_pages):
        #get coins for this api fetch
        queryString = createQueryString(pagedCoins[pageNumber])
        pageStats = fetchStats(queryString)
        fetchedCollated.append(pageStats)
        random_number = random.randint(1,5)
        time.sleep(random_number)
        print(current_time,len(fetchedCollated),'pages Fetched...' )
    result = fetchedCollated
    return  result
def removePages(pagedList): #convert output of fetchStatsAll into 1 big list
    result = []
    for n in range(len(pagedList)):
            result.extend(pagedList[n])
    print('Output file contains:', len(result),'records')
    return result
# PROGRAM 
rapidapiKey = fileManagement.JSONImportEnv('x-rapidapi-key',env_path)
pagedCoins = fileManagement.JSONImport('pagedCoins.json',data_path)
pagedStatsOutput = fetchStatsAll(pagedCoins)
statsDictionarys = removePages(pagedStatsOutput)
fileManagement.JSONExport('pagedStats.json',data_path,statsDictionarys)
print(current_time,'All code run in this block')


13:27:47 Imported Env: /Users/paul/Documents/dev/python/crytpoCurrencyAnalysis/env/env.json
13:27:47 Imported file: /Users/paul/Documents/dev/python/crytpoCurrencyAnalysis/data/pagedCoins.json
Total Pages: 2264
13:27:47 1 pages Fetched...
13:27:47 2 pages Fetched...
13:27:47 3 pages Fetched...
Output file contains: 9 records
/Users/paul/Documents/dev/python/crytpoCurrencyAnalysis/data/pagedStats.json
Records Exported: 9 records
13:27:47 Exported file: /Users/paul/Documents/dev/python/crytpoCurrencyAnalysis/data/pagedStats.json File: Overwritten
13:27:47 All code run in this block
